In [10]:
since_date = '2021-07-07'
until_date = '2021-07-08'

In [2]:
import os
import re 


import numpy as np
import pandas as pd
import datetime


                                       # to handle regular expressions
              # to extract the puntuation symbols
import nltk
from nltk.tokenize import word_tokenize          # to divide strings into tokens

import tensorflow as tf            
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
from tensorflow.keras.layers import Embedding
from tensorflow.keras.callbacks import EarlyStopping


import text_proccessing
from text_proccessing import clean_text
from text_proccessing import remove_stopwords
from text_proccessing import lemmatize_text
from text_proccessing import concatenate_text
from text_proccessing import makeglove
from text_proccessing import make_embedding_matrix

import modhelp
from modhelp import train_val_split
from modhelp import geo_df
from modhelp import suggest_nn2

In [3]:
code_dir = os.getcwd()

parent_dir = os.path.dirname(code_dir)
data_dir = os.path.join(parent_dir,"Data")
dsa= os.path.dirname(parent_dir)
tweet_dir = os.path.join(parent_dir,"TweetMap")
directory = os.path.join(data_dir,'splits')


In [4]:
train_data = pd.read_csv(os.path.join(directory, 'InformativenessTrain_Tokenized.csv'),engine='python')
train_data['text']= train_data['text'].astype(str)

In [5]:
start =  datetime.datetime.now() 
print('splitthis started at: {}'.format(start))
train_samples, val_samples, train_labels, val_labels = train_val_split(train_data, 0.25)
end =  datetime.datetime.now() 
print ('Text done {}'.format(end-start))
start =  datetime.datetime.now() 
print('Loading model,embedding and glove commenced at  {}'.format(start))

model = tf.keras.models.load_model(os.path.join(dsa,'model1'))
print("loading vecotor")
path_to_glove_file = os.path.join(dsa,'WordVector','glove.twitter.27B.200d.txt')

print("indexing")
embeddings_index=makeglove(path_to_glove_file)
print("matrix and vectorize")
embedding_matrix, vectorizer = make_embedding_matrix(train_samples, val_samples, embeddings_index)
end =  datetime.datetime.now() 

print("embedded in {}".format(end-start))

splitthis started at: 2021-07-15 22:20:26.723501
Total size of the dataset: 117339.
Training dataset: 88005.
Validation dataset: 29334.
Text done 0:00:00.031653
Loading model,embedding and glove commenced at  2021-07-15 22:20:26.758372
loading vecotor
indexing
matrix and vectorize
Converted 32283 words (22717 misses).
embedded in 0:03:01.926356


In [11]:
twts = pd.read_csv(os.path.join(tweet_dir,'tweets_df_{0}_{1}.csv'.
                              format(since_date.replace('-',''),until_date.replace('-',''))), engine = 'python')
twts['Text'] = twts['Text'].astype(str)


In [12]:
procstart =  datetime.datetime.now()
print("{}: processing ".format(procstart))
twts['ptext'] = twts['Text'].apply(lambda x: clean_text(x))
twts['ptext'] = twts['ptext'].apply(lambda x: word_tokenize(x))
twts['ptext'] = twts['ptext'].apply(lambda x : remove_stopwords(x))
twts['ptext'] = twts['ptext'].apply(lambda x : lemmatize_text(x))
twts['ptext'] = twts['ptext'].apply(lambda x : concatenate_text(x))
procend=  datetime.datetime.now() 
print("{}: processed in {}".format(procend, procend - procstart))



2021-07-16 00:21:44.993746: processing 
2021-07-16 00:29:29.079660: processed in 0:07:44.085914


In [13]:

predstart = datetime.datetime.now() 
print("{}: predict".format(predstart))
predictions = suggest_nn2(twts, model,vectorizer)

submission_data = {"ID": twts['TweetId'].tolist(),"tweet": twts['Text'].tolist(), "target": predictions}

submission_df = pd.DataFrame(submission_data)
result = twts.join(submission_df.target)
predend = datetime.datetime.now() 
print("predicted {}".format(predend-predstart))
# 
result_inf = result[result['target']==0]
print("working over {} informative tweets".format(len(result_inf)))



2021-07-16 00:29:30.163855: predict
predicted 0:02:56.041242
working over 49177 informative tweets


In [14]:


print("Run geoprocessing over tweets")

geostart =  datetime.datetime.now() 
print("{}: geostart".format(geostart))
df_js = geo_df(result_inf)
geoend =  datetime.datetime.now()
print("{}: geoend {}".format(geoend,geoend-geostart))
i
df_js.to_csv(os.path.join(tweet_dir,'result_{0}_{1}.csv'.
                              format(since_date.replace('-',''),until_date.replace('-',''))))
print("fín")

Run geoprocessing over tweets
2021-07-16 00:32:27.447666: geostart
Models path: /Volumes/Elements/DataScience/anaconda3/envs/machinelearning/lib/python3.8/site-packages/mordecai/models/
You may be using an outdated Geonames index/Mordecai version.
Your index is from 2018-06-05, while your Mordecai version is from 2020-07-11. Please see
https://github.com/openeventdata/mordecai/ for instructions on updating.


  0%|          | 7/49177 [00:27<52:52:54,  3.87s/it]


KeyboardInterrupt: 